In [2]:
using Random
using DifferentialEquations
using Optim
using Plots

# ---------------------------------------------------------------
# 1) Define Lotka-Volterra ODE
# ---------------------------------------------------------------
# We'll treat (alpha, beta, gamma, delta) as parameters
function lotka_volterra!(du, u, p, t)
    alpha, beta, gamma, delta = p
    x, z = u
    du[1] = alpha*x - beta*x*z      # dx/dt
    du[2] = delta*x*z - gamma*z     # dz/dt
end

# ---------------------------------------------------------------
# 2) Generate "true" data with a standard solver (fine or adaptive)
# ---------------------------------------------------------------
function generate_data(alpha_true, beta_true, gamma_true, delta_true, tdata, y0; noise_std=0.01)
    # p = (alpha, beta, gamma, delta)
    p = (alpha_true, beta_true, gamma_true, delta_true)
    prob = ODEProblem(lotka_volterra!, y0, (tdata[1], tdata[end]), p)

    # Solve adaptively with low tolerances, then sample solution at tdata
    sol = solve(prob, Tsit5(); reltol=1e-8, abstol=1e-8)
    y_clean = [sol(ti) for ti in tdata]  # each element is a 2D state

    # Convert to matrix for convenience
    y_clean_mat = hcat(y_clean...)'  # shape (length(tdata), 2)

    # Add noise
    y_noisy = y_clean_mat .+ noise_std .* randn.() .* ones(size(y_clean_mat))
    return y_noisy
end

# ---------------------------------------------------------------
# 3) Fixed-step Euler integrator (we specify the number of points)
# ---------------------------------------------------------------
function euler_integration(p, t0, tf, y0, n_points)
    # p: tuple of parameters (alpha, beta, gamma, delta)
    # n_points in time => dt = (tf - t0) / (n_points - 1)
    dt = (tf - t0)/(n_points - 1)
    t_sol = range(t0, tf, length=n_points)
    dim = length(y0)
    y_sol = zeros(n_points, dim)
    y_sol[1, :] = y0

    for i in 1:(n_points-1)
        # Forward Euler: u_{i+1} = u_i + dt * f(t_i, u_i)
        t_i = t_sol[i]
        x_i = y_sol[i, :]
        # Evaluate ODE:
        du = similar(x_i)
        lotka_volterra!(du, x_i, p, t_i)
        y_sol[i+1, :] = x_i .+ dt .* du
    end

    return collect(t_sol), y_sol
end

# ---------------------------------------------------------------
# 4) Define cost function (sum of squared errors) for parameter fitting
#    No interpolation: we assume t_data matches an index subset in t_euler
#    or we do nearest index. Below we'll do nearest index for general case.
# ---------------------------------------------------------------
function cost_function(params, t_data, data, y0, n_euler_points)
    alpha_est, beta_est, gamma_est, delta_est = params
    p_est = (alpha_est, beta_est, gamma_est, delta_est)

    t0, tf = first(t_data), last(t_data)
    t_eul, y_eul = euler_integration(p_est, t0, tf, y0, n_euler_points)

    residual_sum = 0.0
    for i in eachindex(t_data)
        t_i = t_data[i]
        # Find nearest index in t_eul
        idx = findmin(abs.(t_eul .- t_i))[2]
        residual_sum += sum((y_eul[idx, :] .- data[i, :]).^2)
    end
    return residual_sum
end


cost_function (generic function with 1 method)

In [4]:
# ---------------------------------------------------------------
# 5) Demonstration of the entire pipeline
# ---------------------------------------------------------------

# True parameters
true_params = [1.0, 0.1, 0.5, 0.05]

alpha_true, beta_true, gamma_true, delta_true = true_params

# We'll have 20 data points, from t=0..19
n_data = 50
t_data = range(0, 19, length=n_data)

# Initial condition
y0 = [30, 9]

# Generate noisy data
data_noisy = generate_data(alpha_true, beta_true, gamma_true, delta_true,
                           collect(t_data), y0; noise_std=0.0)

# List of Euler points to test
euler_points_list = [20, 40, 60, 80, 100, 200]

# We'll store the "best" result for each n_euler (lowest cost among 20 seeds)
best_params_dict = Dict{Int,Tuple{Float64,Float64,Float64,Float64}}()
best_cost_dict   = Dict{Int,Float64}()

# -----------------------------------------------------------
# For each n_euler, do 20 random seeds -> pick the best
# -----------------------------------------------------------
for n_euler in euler_points_list
    println("\n### n_euler = $n_euler ###")

    best_cost = Inf
    best_params = (0.0, 0.0, 0.0, 0.0)

    for trial in 1:100
        # Use a different random seed each trial
        Random.seed!(trial)

        # We'll pick an initial guess randomly near (1, 0.5, 0.8, 0.3)
        # Or you can pick any strategy
        init_guess = [max(0.001,alpha_true + 0.1*randn()), max(0.001,beta_true + 0.1*randn()),
                      max(0.001,gamma_true + 0.1*randn()), max(0.001,delta_true + 0.1*randn())]

        # We'll do an unconstrained optimization using Optim
        # cost_function(...) expects a vector for `params`
        f_to_min(x) = cost_function(x, collect(t_data), data_noisy, y0, n_euler)

        
        
        
        # We can use e.g. Nelder-Mead or BFGS
        lower_bounds = fill(0.0, length(init_guess))
        upper_bounds = fill(500, length(init_guess))  # No upper bound (positive values only)
        opts = Optim.Options(iterations=1000)
        # res = optimize(f_to_min, init_guess, NelderMead(), opts)
        # res = optimize(f_to_min, lower_bounds, upper_bounds, init_guess, Fminbox(NelderMead()), opts)
        res = optimize(f_to_min, lower_bounds, upper_bounds, init_guess, Fminbox(SimulatedAnnealing()), opts)
        
        # res = optimize(f_to_min, lower_bounds, upper_bounds, init_guess, Fminbox(BFGS()), opts)
        

        
        this_cost = Optim.minimum(res)
        this_params = Optim.minimizer(res)

        # println("Trial $trial => cost=$this_cost, params=$(this_params)")

        if this_cost < best_cost
            best_cost = this_cost
            best_params = (this_params[1], this_params[2], this_params[3], this_params[4])
        end
    end

    println("Best cost for n_euler=$n_euler is $best_cost with params $best_params")
    best_params_dict[n_euler] = best_params
    best_cost_dict[n_euler]   = best_cost
end

# -----------------------------------------------------------
# 6) Plot only the best (lowest cost) solution for each n_euler
# -----------------------------------------------------------
nplots = length(euler_points_list)
plt = plot(layout=(1,nplots), size=(1200,400), legend=:best)

 # n_data_sim = 100
 #        t_sim = np.linspace(0, 19, n_data_sim)  # from 0 to 19 (inclusive)
        
 #        # ---------------------------------------------------------------
 #        # Generate synthetic "true" data
 #        # (Using solve_ivp with small tolerances as a 'standard solver')
 #        # ---------------------------------------------------------------
 #        print("readhed her")
 #        y_sim = generate_data(a_est, b_est, g_est, d_est,
 #                           t_sim, y0, noise_std=0)


for (i, n_euler) in enumerate(euler_points_list)
    t_best_sim = LinRange(0, 19, 400)
    t_best_sim_range = range(0, 19, length=400)
    
    y_best_sim = generate_data(best_params_dict[n_euler][1], best_params_dict[n_euler][2], best_params_dict[n_euler][3],
        best_params_dict[n_euler][4], collect(t_best_sim_range), y0; noise_std=0.0)
    p_est = best_params_dict[n_euler]
    # t_eul, y_eul = euler_integration(p_est, 0.0, 19.0, y0, n_euler)

    plot!(plt[i], t_data, data_noisy[:,1], seriestype=:scatter,
          marker=:circle, label="Data x", alpha=0.7)
    plot!(plt[i], t_data, data_noisy[:,2], seriestype=:scatter,
          marker=:diamond, label="Data z", alpha=0.7)
    plot!(plt[i], t_best_sim, y_best_sim[:,1], label="Euler x")
    plot!(plt[i], t_best_sim, y_best_sim[:,2], label="Euler z")
    title!(plt[i], "n_euler = $n_euler (best run)")
    xlabel!(plt[i], "Time")
    ylabel!(plt[i], "Population")
end

display(plt)



### n_euler = 20 ###


LoadError: MethodError: no method matching reset!(::SimulatedAnnealing{typeof(Optim.default_neighbor!), typeof(Optim.log_temperature)}, ::Optim.SimulatedAnnealingState{Vector{Float64}, Float64}, ::Optim.BarrierWrapper{OnceDifferentiable{Float64, Vector{Float64}, Vector{Float64}}, Optim.BoxBarrier{Vector{Float64}, Vector{Int64}}, Float64, Float64, Vector{Float64}}, ::Vector{Float64})

[0mClosest candidates are:
[0m  reset!(::Any, [91m::Optim.GradientDescentState[39m, ::Any, ::Any)
[0m[90m   @[39m [36mOptim[39m [90m~/.julia/packages/Optim/HvjCd/src/multivariate/solvers/first_order/[39m[90m[4mgradient_descent.jl:46[24m[39m
[0m  reset!(::Any, [91m::Optim.BFGSState[39m, ::Any, ::Any)
[0m[90m   @[39m [36mOptim[39m [90m~/.julia/packages/Optim/HvjCd/src/multivariate/solvers/first_order/[39m[90m[4mbfgs.jl:69[24m[39m
[0m  reset!(::Any, [91m::Optim.LBFGSState[39m, ::Any, ::Any)
[0m[90m   @[39m [36mOptim[39m [90m~/.julia/packages/Optim/HvjCd/src/multivariate/s

In [21]:
    t_best_sim = range(0, 19, length=400)

generate_data(best_params_dict[20][1], best_params_dict[20][2], best_params_dict[20][3],
        best_params_dict[20][4], collect(t_best_sim), y0; noise_std=0.0)

400×2 Matrix{Float64}:
 1.0       0.5
 1.02855   0.500083
 1.05791   0.500495
 1.08809   0.501245
 1.11908   0.502346
 1.15091   0.503808
 1.18358   0.505644
 1.21708   0.507868
 1.25142   0.510495
 1.28661   0.513541
 1.32262   0.517022
 1.35947   0.520958
 1.39714   0.525368
 ⋮         
 0.420299  0.648209
 0.430088  0.639705
 0.440237  0.631456
 0.450755  0.623459
 0.461654  0.615714
 0.472945  0.608217
 0.48464   0.600969
 0.49675   0.593968
 0.509287  0.587212
 0.522264  0.580701
 0.535694  0.574435
 0.54959   0.568412

In [25]:
max

3

In [8]:
a = [1,2,3,4]

4-element Vector{Int64}:
 1
 2
 3
 4

In [9]:
c,d,e,f = a

4-element Vector{Int64}:
 1
 2
 3
 4

In [11]:
d

2